In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

In [14]:
funda_data = pd.read_csv('Fundamental_data.csv')
flow_data = funda_data.loc[:,['qdate','pcf','PEG_trailing','TICKER']]
funda_data = funda_data.drop(columns=['public_date',"pcf",'staff_sale','PEG_trailing','gvkey','permno','adate'])
funda_data = funda_data.drop_duplicates()
stock_list = ['AAPL','AMZN','BRK-B','GOOG','JNJ','META','MSFT','NVDA','TSLA','V']

,qdate,pcf,PEG_trailing,TICKER
0,12/31/2009,11.996,0.910,MSFT
1,12/31/2009,12.243,0.930,MSFT
2,12/31/2009,12.767,0.970,MSFT
3,03/31/2010,10.135,1.119,MSFT
4,03/31/2010,8.925,0.998,MSFT
...,...,...,...,...
1160,06/30/2019,15.214,-0.102,TSLA
1161,06/30/2019,19.918,-0.133,TSLA
1162,09/30/2019,26.854,-5.117,TSLA
1163,09/30/2019,34.203,-6.488,TSLA


In [ ]:
for s in stock_list:
    funda_stock = funda_data.loc[funda_data.TICKER == s,:]
    funda_stock['qdate'] = funda_data.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
    funda_stock.set_index('qdate')
    sns.heatmap(funda_data.corr().fillna(0))
    plt.title(s)
    plt.show()

In [ ]:
for s in stock_list:
    flow_stock = flow_data.loc[flow_data.TICKER == s,:]
    flow_stock['qdate'] = flow_stock.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
    flow_stock.set_index('qdate')
    print(flow_stock.corr().fillna(0))
    sns.heatmap(flow_stock.corr().fillna(0))
    plt.title(s)
    plt.show()

In [27]:
def merge_left(funda_stock,name,flow_data,freq):
    price = pd.read_csv('../stock_price/'+freq+'/'+name+'.csv')
    price['Date'] = price.Date.apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
    close = price.loc[:,['Date']]
    df = pd.merge(funda_stock,close,how='right',left_on=['qdate'],right_on=['Date'])
    df = pd.merge(flow_data,df,how='right',left_on=['qdate'],right_on=['Date'])
# Here we merge the dataset together using the forward filling method, which is using the old time data to fill the later time.
    df.fillna(method='ffill',inplace=True)
    df.set_index('Date',inplace=True)
    df.drop(columns=['qdate_x','qdate_y','TICKER_y','TICKER_x'],inplace=True)
    df.dropna(inplace=True,how='all')
    df.to_csv('FundaIndicators/'+freq+'/'+name+'.csv')

In [28]:
for name in pd.unique(funda_data.TICKER).tolist():
    funda_stock = funda_data.loc[funda_data.TICKER==name,:]
    funda_stock['qdate'] = funda_data.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
    flow_stock = flow_data.loc[flow_data.TICKER==name,:]
    flow_stock['qdate'] = flow_stock.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
    if name == 'FB':
        name = 'META'
    elif name == 'BRK':
        name = 'BRK-B'
    elif name == 'GOOGL':
        name = 'GOOG'
    merge_left(funda_stock,name,flow_stock,'Daily')
    merge_left(funda_stock,name,flow_stock,'Weekly')
    merge_left(funda_stock,name,flow_stock,'Monthly')

In [ ]:
freq = 'Daily'

for name in stock_list:
    FundaInd = pd.read_csv('FundaIndicators/'+freq+'/'+name+'.csv')
    sns.heatmap(FundaInd.corr().fillna(0))
    plt.title(name)
    plt.show()

In [ ]:
freq = 'Weekly'

for name in stock_list:
    FundaInd = pd.read_csv('FundaIndicators/'+freq+'/'+name+'.csv')
    sns.heatmap(FundaInd.corr().fillna(0))
    plt.title(name)
    plt.show()



In [ ]:
freq = 'Monthly'

for name in stock_list:
    FundaInd = pd.read_csv('FundaIndicators/'+freq+'/'+name+'.csv')
    sns.heatmap(FundaInd.corr().fillna(0))
    plt.title(name)
    plt.show()